In [4]:
!pwd

/home/sagemaker-user/Brain-Tumor-Classification-MRI/notebooks


In [5]:
%cd ..

/home/sagemaker-user/Brain-Tumor-Classification-MRI


# Training Configuration

In [6]:
from dataclasses import dataclass

In [7]:
@dataclass
class TrainingConfig:
    training_folder: str
    trained_model_path: str
    updated_model_path: str
    training_data: str
    params_epochs: int
    params_batch_size: int
    params_augmentation: bool
    params_image_size: list

# Configuration Manager

In [8]:
from src.config import ConfigManager
from src.constants import *
import os

In [9]:
class ConfigManager(ConfigManager):
    def get_training_config(self):
        training_config = self.config.training
        model_config = self.config.model
        training_data_path = os.path.join(
            self.artifacts_folder, self.config.data_ingestion.data_folder_name, 'Training'
        )
        
        return TrainingConfig(
            training_folder = os.path.join(self.artifacts_folder, training_config.folder_name),
            trained_model_path = os.path.join(self.artifacts_folder, training_config.trained_model_path),
            updated_model_path = os.path.join(self.artifacts_folder, self.config.model.updated_model_path),
            training_data = training_data_path,
            params_epochs = self.params.EPOCHS,
            params_batch_size = self.params.BATCH_SIZE,
            params_augmentation = self.params.AUGMENTATION,
            params_image_size = self.params.IMAGE_SIZE
        )

# Train Function

In [7]:
import tensorflow as tf

2024-09-27 21:15:30.949057: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
class Train:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def data_generator(self):
        if self.config.params_augmentation:
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1/255.,
                validation_split = 0.27,
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2
            )
        else:
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1/255.,
                validation_split = 0.27
            )
        self.train_generator = train_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size
        )

        val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale = 1/255.,
            validation_split = 0.27
        )
        self.val_generator = val_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = True,
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size
        )

    def fit(self, callbacks: list):
        model = tf.keras.models.load_model(self.config.updated_model_path)
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        val_steps = self.val_generator.samples // self.val_generator.batch_size

        model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = steps_per_epoch,
            validation_data = self.val_generator,
            validation_steps = val_steps,
            callbacks = callbacks
        )

        model.save(self.config.trained_model_path)

# Perform The Step

In [13]:
from src.components.callbacks import Callbacks

In [ ]:
try:
    callbacks_config = ConfigManager().get_callbacks_config()
    callbacks = Callbacks(callbacks_config).get_callbacks()
    
    training_config = ConfigManager().get_training_config()
    train = Train(training_config)
    train.data_generator()
    train.fit(callbacks)
except Exception as e:
    raise e

[2024-09-27 21:22:34,377: INFO: utils: The file: config.yaml loaded successfully...]
[2024-09-27 21:22:34,380: INFO: utils: The file: params.yaml loaded successfully...]
[2024-09-27 21:22:34,381: INFO: utils: Directory 'artifacts' was successfully created...]
[2024-09-27 21:22:34,382: INFO: utils: Directory 'artifacts/callbacks/tensorboard' was successfully created...]
[2024-09-27 21:22:34,382: INFO: utils: Directory 'artifacts/callbacks/checkpoint' was successfully created...]
[2024-09-27 21:22:34,386: INFO: utils: The file: config.yaml loaded successfully...]
[2024-09-27 21:22:34,388: INFO: utils: The file: params.yaml loaded successfully...]
[2024-09-27 21:22:34,389: INFO: utils: Directory 'artifacts' was successfully created...]
Found 2097 images belonging to 4 classes.
Found 773 images belonging to 4 classes.
Epoch 1/5


2024-09-27 21:22:36.672418: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-09-27 21:22:37.588460: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-09-27 21:22:39.490783: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-09-27 21:22:39.606008: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-09-27 21:22:40.256881: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


# Tensorboard Logs

#### If you want to see the logs, run the following terminal command:
`tensorboard --logdir artifacts/callbacks/tensorboard`